The main objective of the following article is to obtain a step-by-step guide on how to forecast with `StackingRegressor` for a *time series model with machine learning* using `Mlforecast`.

During this walkthrough, we will become familiar with the main `MlForecast` class and some relevant methods such as `mlforecast.fit`, `mlforecast.predict` and `mlforecast.cross_validation` in other.

Let's start!!!

<a class="anchor" id="0.1"></a>

1.	[Introduction](#1)
2.	[What is StackingRegressor?](#2)
3.	[Loading libraries and data](#3)
4.	[Explore Data with the plot method](#4)
5.	[Modeling with MLForecast](#5)
6.  [Feature importances](#6)
7.  [Evaluate the model’s performance](#7)
8.  [Evaluate the model](#8)
9.  [References](#9)

# **Introduction** <a class="anchor" id="1"></a>

[Table of Contents](#0.1)

`StackingRegressor` is a class in the `scikit-learn Python` library that implements the assembly technique known as stacking for regression problems. `Stacking` is a strategy that combines multiple regression models to obtain more accurate predictions and improve overall model performance.

The idea behind `StackingRegressor` is to train several base regression models on the data set and then combine their predictions using a final regressor (metaestimator). The base regression models can be different regression algorithms, such as linear regression, decision trees, ridge regression, etc.

# **What is StackingRegressor?** <a class="anchor" id="2"></a>
[Table of Contents](#0.1)

Stacking is an assembly technique that combines multiple regression models to obtain more accurate predictions. Instead of relying on a single regression model, stacking uses a meta model (also known as a metaestimator) that learns how to combine the predictions of the base regression models.

The basic process of regression stacking is as follows:

1. Select and train several base regression models using different algorithms or configurations.
2. Use the base regression models to make predictions on the training set.
3. Use these predictions as features (inputs) for the target model (final regressor).
4. Train the meta model on the generated features and the real labels corresponding to the training set.
5. Use the meta model to make predictions on new data.

The main idea is that the base regression models capture different aspects of the data set and, by combining their predictions using the meta model, a more robust and generalizable model is obtained.

It should be noted that it is important to take into account the diversity and complementarity of the base models for stacking to be effective. Selecting models that have different strengths and weaknesses helps reduce bias and improve the generalizability of the final model.

StackingRegressor combines the predictions of the base regression models using the final regressor, which learns how to combine the predictions to obtain a more accurate final output.

It is important to note that, as with any machine learning model, hyperparameters must be tuned and proper validation performed to obtain the best results. Additionally, StackingRegressor also supports the ability to use feature transformations on each base model, which can further improve performance.


# **Loading libraries and data** <a class="anchor" id="3"></a>

[Table of Contents](#0.1)

In [1]:
# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

import scipy.stats as stats

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime
import time
from datetime import datetime, timedelta

# 
# ==============================================================================
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.seasonal import seasonal_decompose 

In [2]:
# Models Sklearn
# ==============================================================================
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# MLForecast
# ==============================================================================
from mlforecast import MLForecast
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
from window_ops.ewm import ewm_mean
from mlforecast.target_transforms import Differences
from utilsforecast.plotting import plot_series
from mlforecast.utils import PredictionIntervals
from mlforecast.utils import generate_daily_series, generate_prices_for_series

In [3]:
# Plot
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('grayscale') # fivethirtyeight  grayscale  classic
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#003333',  # #212946 #008080
    'axes.facecolor': '#008080',
    'savefig.facecolor': '#008080',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#000000',  #2A3459
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)
# Define the plot size
# ==============================================================================

plt.rcParams['figure.figsize'] = (18,7)

# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")

## **Read Data**

We will use [real sales data](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data) made available by Favorita, a large Ecuadorian grocery chain.

We have sales data from 2013 to 2017 for multiple stores and product categories.

In [4]:
data=pd.read_csv("train.csv",index_col='id', parse_dates=['date'])
data.head()

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


In [5]:
data2 = data.loc[(data['store_nbr'] == 1) & (data['family'].isin(['MEATS', 'PERSONAL CARE'])), ['date', 'family', 'sales']]
data2

,date,family,sales
id,,,
24,2013-01-01,MEATS,0.000
25,2013-01-01,PERSONAL CARE,0.000
1806,2013-01-02,MEATS,369.101
1807,2013-01-02,PERSONAL CARE,194.000
3588,2013-01-03,MEATS,272.319
...,...,...,...
2995567,2017-08-13,PERSONAL CARE,45.000
2997348,2017-08-14,MEATS,192.763
2997349,2017-08-14,PERSONAL CARE,159.000


In [6]:
dec25 = list()
for year in range(2013,2017):
    for family in ['MEATS', 'PERSONAL CARE']:
        dec18 = data2.loc[(data2['date'] == f'{year}-12-18') & (data2['family'] == family)]
        dec25 += [{'date': pd.Timestamp(f'{year}-12-25'), 'family': family, 'sales': dec18['sales'].values[0]}]
data2 = pd.concat([data2, pd.DataFrame(dec25)], ignore_index=True).sort_values('date')
data2

,date,family,sales
0,2013-01-01,MEATS,0.000
1,2013-01-01,PERSONAL CARE,0.000
2,2013-01-02,MEATS,369.101
3,2013-01-02,PERSONAL CARE,194.000
4,2013-01-03,MEATS,272.319
...,...,...,...
3363,2017-08-13,PERSONAL CARE,45.000
3364,2017-08-14,MEATS,192.763
3365,2017-08-14,PERSONAL CARE,159.000
3366,2017-08-15,MEATS,274.176


The input to MlForecast is always a data frame in long format with three columns: unique_id, ds and y:

* The `unique_id` (string, int or category) represents an identifier for the series.

* The `ds` (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

* The `y` (numeric) represents the measurement we wish to forecast.

In [7]:
data2 = data2.rename(columns={'date': 'ds', 'sales': 'y', 'family': 'unique_id'})
data2

,ds,unique_id,y
0,2013-01-01,MEATS,0.000
1,2013-01-01,PERSONAL CARE,0.000
2,2013-01-02,MEATS,369.101
3,2013-01-02,PERSONAL CARE,194.000
4,2013-01-03,MEATS,272.319
...,...,...,...
3363,2017-08-13,PERSONAL CARE,45.000
3364,2017-08-14,MEATS,192.763
3365,2017-08-14,PERSONAL CARE,159.000
3366,2017-08-15,MEATS,274.176


In [8]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3376 entries, 0 to 3367
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         3376 non-null   datetime64[ns]
 1   unique_id  3376 non-null   object        
 2   y          3376 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 105.5+ KB


# **Explore Data with the plot method** <a class="anchor" id="4"></a>

[Table of Contents](#0.1)

We are going to use the `plot_series` function to visualize our data.

In [35]:
fig=plot_series(data2)
fig.savefig("../tutorials/stacking_regressor_forecasting__eda.png")

![](../tutorials/stacking_regressor_forecasting__eda.png)

## **The Augmented Dickey-Fuller Test**
An Augmented Dickey-Fuller (ADF) test is a type of statistical test that determines whether a unit root is present in time series data. Unit roots can cause unpredictable results in time series analysis. A null hypothesis is formed in the unit root test to determine how strongly time series data is affected by a trend. By accepting the null hypothesis, we accept the evidence that the time series data is not stationary. By rejecting the null hypothesis or accepting the alternative hypothesis, we accept the evidence that the time series data is generated by a stationary process. This process is also known as stationary trend. The values of the ADF test statistic are negative. Lower ADF values indicate a stronger rejection of the null hypothesis.

Augmented Dickey-Fuller Test is a common statistical test used to test whether a given time series is stationary or not. We can achieve this by defining the null and alternate hypothesis.

- Null Hypothesis: Time Series is non-stationary. It gives a time-dependent trend.
- Alternate Hypothesis: Time Series is stationary. In another term, the series doesn’t depend on time.

- ADF or t Statistic < critical values: Reject the null hypothesis, time series is stationary.
- ADF or t Statistic > critical values: Failed to reject the null hypothesis, time series is non-stationary.

In [10]:
def augmented_dickey_fuller_test(series , column_name):
    print (f'Dickey-Fuller test results for columns: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of observations used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("The data is stationary")
    else:
        print("Conclusion:====>")
        print("The null hypothesis cannot be rejected")
        print("The data is not stationary")

In [11]:
augmented_dickey_fuller_test(data2["y"],"sales")

Dickey-Fuller test results for columns: sales
Test Statistic                -7.248511e+00
p-value                        1.805637e-10
No Lags Used                   2.900000e+01
Number of observations used    3.346000e+03
Critical Value (1%)           -3.432306e+00
Critical Value (5%)           -2.862404e+00
Critical Value (10%)          -2.567230e+00
dtype: float64
Conclusion:====>
Reject the null hypothesis
The data is stationary


## **Autocorrelation plots**

### **Autocorrelation Function**

**Definition 1.** Let $\{x_t;1 ≤ t ≤ n\}$ be a time series sample of size n from $\{X_t\}$.
1. $\bar x = \sum_{t=1}^n \frac{x_t}{n}$ is called the sample mean of $\{X_t\}$.
2. $c_k =\sum_{t=1}^{n−k} (x_{t+k}- \bar x)(x_t−\bar x)/n$ is known as the sample autocovariance function of $\{X_t\}$.
3. $r_k = c_k /c_0$ is said to be the sample autocorrelation function of $\{X_t\}$. 

Note the following remarks about this definition:
 
* Like most literature, this guide uses ACF to denote the sample autocorrelation function as well as the autocorrelation function. What is denoted by ACF can easily be identified in context.

* Clearly c0 is the sample variance of $\{X_t\}$. Besides, $r_0 = c_0/c_0 = 1$ and for any integer $k, |r_k| ≤ 1$.

* When we compute the ACF of any sample series with a fixed length $n$, we cannot put too much confidence in the values of $r_k$ for large k’s, since fewer pairs of $(x_{t +k }, x_t )$ are available for calculating $r_k$ as $k$ is large. One rule of thumb is not to estimate $r_k$ for $k > n/3$, and another is $n ≥ 50, k ≤ n/4$. In any case, it is always a good idea to be careful.

* We also compute the ACF of a nonstationary time series sample by Definition 1. In this case, however, the ACF or $r_k$ very slowly or hardly tapers off as $k$ increases.

* Plotting the ACF $(r_k)$ against lag $k$ is easy but very helpful in analyzing time series sample. Such an ACF plot is known as a correlogram.

* If $\{X_t\}$ is stationary with $E(X_t)=0$ and $\rho_k =0$ for all $k \neq 0$,thatis,itisa white noise series, then the sampling distribution of $r_k$ is asymptotically normal with the mean 0 and the variance of $1/n$. Hence, there is about 95% chance that $r_k$ falls in the interval $[−1.96/√n, 1.96/√n]$.

Now we can give a summary that (1) if the time series plot of a time series clearly shows a trend or/and seasonality, it is surely nonstationary; (2) if the ACF $r_k$ very slowly or hardly tapers off as lag $k$ increases, the time series should also be nonstationary.

In [12]:
fig, axs = plt.subplots(nrows=1, ncols=2)

plot_acf(data2["y"],  lags=30, ax=axs[0],color="fuchsia")
axs[0].set_title("Autocorrelation");

# Grafico
plot_pacf(data2["y"],  lags=30, ax=axs[1],color="lime")
axs[1].set_title('Partial Autocorrelation')
plt.savefig("../tutorials/stacking_regressor_forecasting__autocorrelation.png")
plt.close();

![](../tutorials/stacking_regressor_forecasting__autocorrelation.png)

## **Decomposition of the time series**

How to decompose a time series and why?

In time series analysis to forecast new values, it is very important to know past data. More formally, we can say that it is very important to know the patterns that values follow over time. There can be many reasons that cause our forecast values to fall in the wrong direction. Basically, a time series consists of four components. The variation of those components causes the change in the pattern of the time series. These components are:

* **Level:** This is the primary value that averages over time.
* **Trend:** The trend is the value that causes increasing or decreasing patterns in a time series.
* **Seasonality:** This is a cyclical event that occurs in a time series for a short time and causes short-term increasing or decreasing patterns in a time series.
* **Residual/Noise:** These are the random variations in the time series.

Combining these components over time leads to the formation of a time series. Most time series consist of level and noise/residual and trend or seasonality are optional values.

If seasonality and trend are part of the time series, then there will be effects on the forecast value. As the pattern of the forecasted time series may be different from the previous time series.

The combination of the components in time series can be of two types:
* Additive
* multiplicative

### Additive

In [13]:
a = seasonal_decompose(data2["y"], model = "additive", period=24).plot()
a.savefig('../tutorials/stacking_regressor_forecasting__seasonal_decompose_aditive.png')
plt.close()

![](../tutorials/stacking_regressor_forecasting__seasonal_decompose_aditive.png)

### Multiplicative

In [14]:
m = seasonal_decompose(data2["y"], model = "Multiplicative", period=24).plot()
m.savefig('../tutorials/stacking_regressor_forecasting__seasonal_decompose_multiplicative.png')
plt.close();

![](../tutorials/stacking_regressor_forecasting__seasonal_decompose_multiplicative.png)

# **Modeling with MLForecast** <a class="anchor" id="5"></a>

[Table of Contents](#0.1)

## **Building Model**

`Stacked` generalization is a method for combining estimators to reduce their biases. More precisely, the predictions of each individual estimator are stacked together and used as input to a final estimator to compute the prediction. This final estimator is trained through cross-validation.

The StackingRegressor provide such strategies which can be applied to  regression problems.

The estimators parameter corresponds to the list of the estimators which are stacked together in parallel on the input data. It should be given as a list of names and estimators:

In [15]:
estimators = [('ridge', RidgeCV()),
              ('lasso', LassoCV(random_state=42)),
              ('knr', KNeighborsRegressor(n_neighbors=20,
                                          metric='euclidean'))]

The `final_estimator` will use the predictions of the estimators as input. It needs to be a regressor when using  `StackingRegressor`, respectively:

In [16]:

final_estimator = GradientBoostingRegressor(
    n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,random_state=42)

In [17]:
regressor=[StackingRegressor(estimators=estimators,final_estimator=final_estimator)]

We fit the models by instantiating a new `MlForecast` object with the following parameters:

* `models:` a list of models. Select the models you want from models and import them.

* `freq:` a string indicating the frequency of the data. (See [panda’s available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

* `lags:` Lags of the target to uses as feature.

* `lag_transforms:` Mapping of target lags to their transformations.

* `date_features:` Features computed from the dates. Can be `pandas` date attributes or functions that will take the dates as input.

* `differences:` Differences to take of the target before computing the features. These are restored at the forecasting step.

* `num_threads:` Number of threads to use when computing the features.

* `target_transforms:` Transformations that will be applied to the target computing the features and restored after the forecasting step.

Any settings are passed into the constructor. Then you call its fit method and pass in the historical data frame.

In [18]:
def even_day(dates):
    return dates.day % 2 == 0

In [19]:
mlf = MLForecast(models=regressor,
                 freq='D', 
                 lags=[1,7,14],
                 lag_transforms={1: [expanding_mean], 12: [(rolling_mean, 7)] },
                 date_features=['dayofweek', 'month', even_day],
                 num_threads=2,        
                 )
mlf

MLForecast(models=[StackingRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'expanding_mean_lag1', 'rolling_mean_lag12_window_size7'], date_features=['dayofweek', 'month', <function even_day at 0x15f9404c0>], num_threads=2)

In [20]:
prep = mlf.preprocess(data2)
prep

,ds,unique_id,y,lag1,lag7,lag14,expanding_mean_lag1,rolling_mean_lag12_window_size7,dayofweek,month,even_day
36,2013-01-19,MEATS,284.48400,519.204,260.07098,328.940,302.905942,257.790571,5,1,False
37,2013-01-19,PERSONAL CARE,92.00000,69.000,103.00000,141.000,106.055556,111.285714,5,1,False
38,2013-01-20,MEATS,108.28600,284.484,131.26900,113.282,301.936366,314.006286,6,1,True
39,2013-01-20,PERSONAL CARE,48.00000,92.000,61.00000,49.000,105.315789,128.285714,6,1,True
40,2013-01-21,MEATS,274.88602,108.286,311.57300,266.720,292.253848,317.013711,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...
3363,2017-08-13,PERSONAL CARE,45.00000,227.000,74.00000,62.000,129.167953,142.857143,6,8,False
3364,2017-08-14,MEATS,192.76300,88.182,236.74600,254.116,313.633686,292.465713,0,8,True
3365,2017-08-14,PERSONAL CARE,159.00000,45.000,187.00000,194.000,129.118031,143.142857,0,8,True
3366,2017-08-15,MEATS,274.17600,192.763,226.22499,336.499,313.562037,292.842570,1,8,False


## **Fit method**

The fit method uses the following parameters: 

|Parameters |Type |Default |Details| 
|-----------|-------|-----------|-------| 
|df| DataFrame| |Series data in long format.| 
|target_col |str |y |Column that contains the target. 
|prediction_intervals |typing.Optional[mlforecast.utils.PredictionIntervals] |None |Configuration to calibrate prediction intervals (Conformal Prediction).| 
|fitted |bool |False |Save in-sample predictions.| 


For more information on the parameters: [you can see it here](https://nixtla.github.io/mlforecast/forecast.html#mlforecast.fit)

In [21]:
# fit the models
mlf.fit(data2, fitted=True)

MLForecast(models=[StackingRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'expanding_mean_lag1', 'rolling_mean_lag12_window_size7'], date_features=['dayofweek', 'month', <function even_day at 0x15f9404c0>], num_threads=2)

Let's see the results of our model in this case the `StackingRegressor model`. We can observe it with the following instruction:

Let us now visualize the fitted values of our models.

In [22]:
result=mlf.forecast_fitted_values()
result=result.set_index("unique_id")
result

,ds,y,StackingRegressor
unique_id,,,
MEATS,2013-01-19,284.48400,284.190771
MEATS,2013-01-20,108.28600,123.421199
MEATS,2013-01-21,274.88602,142.679338
MEATS,2013-01-22,274.26100,77.257890
MEATS,2013-01-23,312.19900,283.073063
...,...,...,...
PERSONAL CARE,2017-08-11,53.00000,95.121827
PERSONAL CARE,2017-08-12,227.00000,262.106993
PERSONAL CARE,2017-08-13,45.00000,165.725083


In [23]:
from statsmodels.stats.diagnostic import normal_ad
from scipy import stats

sw_result = stats.shapiro(result["StackingRegressor"])
ad_result = normal_ad(np.array(result["StackingRegressor"]), axis=0)
dag_result = stats.normaltest(result["StackingRegressor"], axis=0, nan_policy='propagate')

It's important to note that we can only use this method if we assume that the residuals of our validation predictions are normally distributed. To see if this is the case, we will use a PP-plot and test its normality with the Anderson-Darling, Kolmogorov-Smirnov, and D’Agostino K^2 tests.

The PP-plot(Probability-to-Probability) plots the data sample against the normal distribution plot in such a way that if normally distributed, the data points will form a straight line.

The three normality tests determine how likely a data sample is from a normally distributed population using p-values. The null hypothesis for each test is that "the sample came from a normally distributed population". This means that if the resulting p-values are below a chosen alpha value, then the null hypothesis is rejected. Thus there is evidence to suggest that the data comes from a non-normal distribution. For this article, we will use an Alpha value of 0.01.

In [24]:
result=mlf.forecast_fitted_values()
fig, axs = plt.subplots(nrows=2, ncols=2)

# plot[1,1]
result["StackingRegressor"].plot(ax=axs[0,0])
axs[0,0].set_title("Residuals model");

# plot
#plot(result["StackingRegressor"], ax=axs[0,1]);
axs[0,1].hist(result["StackingRegressor"], density=True,bins=50, alpha=0.5 )
axs[0,1].set_title("Density plot - Residual");

# plot
stats.probplot(result["StackingRegressor"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')
axs[1,0].annotate("SW p-val: {:.4f}".format(sw_result[1]), xy=(0.05,0.9), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("AD p-val: {:.4f}".format(ad_result[1]), xy=(0.05,0.8), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("DAG p-val: {:.4f}".format(dag_result[1]), xy=(0.05,0.7), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))
# plot
plot_acf(result["StackingRegressor"],  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation");

plt.savefig("../tutorials/stacking_regressor_forecasting__plot_residual_model.png")
plt.close();

![](../tutorials/stacking_regressor_forecasting__plot_residual_model.png)

## **Predict method**

* h (int)		Number of periods to predict.

For more information on the parameters: [you can see it here](https://nixtla.github.io/mlforecast/forecast.html#mlforecast.predict)

In [25]:
forecast_df = mlf.predict(h=30) 
forecast_df

,unique_id,ds,StackingRegressor
0,MEATS,2017-08-16,297.809481
1,MEATS,2017-08-17,273.566064
2,MEATS,2017-08-18,591.610487
3,MEATS,2017-08-19,294.983679
4,MEATS,2017-08-20,131.962984
5,MEATS,2017-08-21,256.168284
6,MEATS,2017-08-22,262.271360
7,MEATS,2017-08-23,292.702878
8,MEATS,2017-08-24,278.309116
9,MEATS,2017-08-25,608.217098


## **Plot prediction**

In [26]:
gif=plot_series(data2, forecast_df, max_insample_length=200)
fig.savefig('../tutorials/stacking_regressor_forecasting__plot_forecasting.png')

![](../tutorials/stacking_regressor_forecasting__plot_forecasting_intervals.png)

# **Evaluate the model’s performance** <a class="anchor" id="7"></a>

[Table of Contents](#0.1)

In previous steps, we’ve taken our historical data to predict the future. However, to asses its accuracy we would also like to know how the model would have performed in the past. To assess the accuracy and robustness of your models on your data perform `Cross-Validation`.

With time series data, `Cross Validation` is done by defining a sliding window across the historical data and predicting the period following it. This form of `cross-validation` allows us to arrive at a better estimation of our model’s predictive abilities across a wider range of temporal instances while also keeping the data in the training set contiguous as is required by our models.

The following graph depicts such a `Cross Validation` Strategy:

![](https://raw.githubusercontent.com/Nixtla/statsforecast/main/nbs/imgs/ChainedWindows.gif)

## **Perform time series cross-validation**

In order to get an estimate of how well our model will be when predicting future data we can perform cross validation, which consist on training a few models independently on different subsets of the data, using them to predict a validation set and measuring their performance.

Since our data depends on time, we make our splits by removing the last portions of the series and using them as validation sets. This process is implemented in `MLForecast.cross_validation`.

`Cross-validation` of time series models is considered a best practice but most implementations are very slow. The `MlForecast` library implements cross-validation as a distributed operation, making the process less time-consuming to perform. If you have big datasets you can also perform `Cross Validation` in a distributed cluster using `Ray, Dask or Spark`.

Depending on your computer, this step should take around 1 min.

The `cross_validation` method from the StatsForecast class takes the following arguments.

* `df:` training data frame

* `h (int):` represents h steps into the future that are being forecasted. In this case, 12 months ahead.

* `step_size (int):` step size between each window. In other words: how often do you want to run the forecasting processes.

* `n_windows(int):` number of windows used for `cross validation`. In other words: what number of forecasting processes in the past do you want to evaluate.

In [27]:
cv_result = mlf.cross_validation(
    data2,
    n_windows=5,  # number of models to train/splits to perform
    window_size=30,  # length of the validation set in each window
)

The crossvaldation_df object is a new data frame that includes the following columns:

* `unique_id:` index. If you dont like working with index just run `crossvalidation_df.resetindex()`.
* `ds:` datestamp or temporal index
* `cutoff:` the last datestamp or temporal index for the `n_windows`.
* `y:` true value
* `model:` columns with the model’s name and fitted value.

In [28]:
cv_result

,unique_id,ds,cutoff,y,StackingRegressor
0,PERSONAL CARE,2017-03-19,2017-03-18,82.000,90.317027
1,MEATS,2017-03-19,2017-03-18,121.816,131.729114
2,PERSONAL CARE,2017-03-20,2017-03-18,159.000,153.983111
3,MEATS,2017-03-20,2017-03-18,223.134,267.823227
4,PERSONAL CARE,2017-03-21,2017-03-18,172.000,161.341794
...,...,...,...,...,...
55,PERSONAL CARE,2017-08-13,2017-07-16,45.000,116.170341
56,MEATS,2017-08-14,2017-07-16,192.763,281.996209
57,PERSONAL CARE,2017-08-14,2017-07-16,159.000,155.011884
58,MEATS,2017-08-15,2017-07-16,274.176,285.118047


In [29]:
def plot_cv(df, df_cv, uid, fname, last_n=24 * 14):
    cutoffs = df_cv.query('unique_id == @uid')['cutoff'].unique()
    fig, ax = plt.subplots(nrows=len(cutoffs), ncols=1, figsize=(14, 6), gridspec_kw=dict(hspace=0.8))
    for cutoff, axi in zip(cutoffs, ax.flat):
        df.query('unique_id == @uid').tail(last_n).set_index('ds').plot(ax=axi, title=uid, y='y')
        df_cv.query('unique_id == @uid & cutoff == @cutoff').set_index('ds').plot(ax=axi, title=uid, y='StackingRegressor')
    fig.savefig(fname, bbox_inches='tight')
    plt.close()

In [30]:
plot_cv(data2, cv_result, 'MEATS', '../tutorials/stacking_regressor_forecasting__cross_validation_predictions.png')

![](../tutorials/stacking_regressor_forecasting__cross_validation_predictions.png)

# **Evaluate the model** <a class="anchor" id="8"></a>

[Table of Contents](#0.1)

We can now compute the accuracy of the forecast using an appropiate accuracy metric. Here we’ll use the Root Mean Squared Error (RMSE). To do this, we first need to `install datasetsforecast`, a Python library developed **by Nixtla** that includes a function to compute the RMSE.

`pip install datasetsforecast`

In [31]:
from datasetsforecast.losses import rmse

The function to compute the RMSE takes two arguments:

1. The actual values.
2. The forecasts, in this case `StackingRegressor Model`.

In [32]:
from datasetsforecast.losses import mse, mae, rmse

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [33]:
evaluation_df = evaluate_cross_validation(cv_result.set_index("unique_id"), rmse)

evaluation_df

,StackingRegressor,best_model
unique_id,,
MEATS,89.249217,StackingRegressor
PERSONAL CARE,39.130482,StackingRegressor


The result obtained in this case is given by each type of `unique_ide` identifying the model created with the `RMSE` metric, in which both results are compared.

In [34]:
cv_rmse = cv_result.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df['StackingRegressor'])).mean()
print("RMSE using cross-validation: ", cv_rmse)

RMSE using cross-validation:  64.189849303637


# **References** <a class="anchor" id="9"></a>

[Table of Contents](#0)

1. Changquan Huang • Alla Petukhina. Springer series (2022). Applied Time Series Analysis and Forecasting with Python. 
2. Ivan Svetunkov. [Forecasting and Analytics with the Augmented Dynamic Adaptive Model (ADAM)](https://openforecast.org/adam/)
3. [James D. Hamilton. Time Series Analysis Princeton University Press, Princeton, New Jersey, 1st Edition, 1994.](https://press.princeton.edu/books/hardcover/9780691042893/time-series-analysis)
4. [Nixtla Parameters for Mlforecast](https://nixtla.github.io/mlforecast/forecast.html).
5. [Pandas available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
6. [Rob J. Hyndman and George Athanasopoulos (2018). “Forecasting principles and practice, Time series cross-validation”.](https://otexts.com/fpp3/tscv.html).
7. [Seasonal periods- Rob J Hyndman](https://robjhyndman.com/hyndsight/seasonal-periods/).
8. [Catboost](https://catboost.ai)